# Deep Learning Model Evaluation

We are almost always interested in ploting the training accuracy and testing (validation set) accuracy over epochs and how the loss is decreasing over epochs

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

Using TensorFlow backend.


60000 train samples
10000 test samples
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


### Plotting the accuracy/loss over each epoch

In [ ]:
history = model.fit(x_train, y_train, validation_split=0.33, epochs=20, batch_size=128, verbose=0)

# list all data in history
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Keras Callbacks
**Callbacks:** functions that can be applied at certain stages of the training process, such as at the end of each epoch
- Specifically, in our solution, we included EarlyStopping(monitor='val_loss', patience=2) to define that we wanted to monitor the validation loss at each epoch and after the validation loss has not improved after two epochs, training is interrupted
- However, since we set patience=2, we won’t get the best model, but the model two epochs after the best model. Therefore, optionally, we can include a second operation, ModelCheckpoint which saves the model to a file after every checkpoint

![early stopping image](../Images/early_stopping.png)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Set callback functions to early stop training and save the best model so far
callbacks = [EarlyStopping(monitor='val_loss', patience=2),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

# Train neural network
history = model.fit(train_features, # Features
                    train_target, # Target vector
                    epochs=20, # Number of epochs
                    callbacks=callbacks, # Early stopping
                    verbose=0, # Print description after each epoch
                    batch_size=100, # Number of observations per batch
                    validation_data=(test_features, test_target)) # Data for evaluation

### Tensorboard
It hosts a website on your local machine in which you can monitor things like accuracy, cost functions and visualize the computational graph that Tensorflow is running based on what you defined in Keras

- For monitoring progress, open the terminal: tensorboard --logdir Graph

In [2]:
from keras.callbacks import TensorBoard

tensor_board = TensorBoard(log_dir='./Graph')
model.fit(x_train, y_train, verbose=1, callbacks=[tensor_board])

1875/1875 [==============================] - 10s 5ms/step - loss: 0.2241 - accuracy: 0.9338


## Saving Keras Models
Given that deep learning models can take hours, days, or weeks to train, it is paramount to know how to save and load them from disk

- Option 1: Weights + Model Architecture (preferred)
- Option 2: Save/Load the Entire Model

Option 1 is more efficent because it's only storing the weights while Option 2 stores the weights and the architecture

#### Weights + Model Architecture

In [ ]:
# Option 1: Save Weights + Architecture
model.save_weights('model_weights.h5')
with open('model_architecture.json', 'w') as f:
    f.write(model.to_json())
# Option 1: Load Weights + Architecture
with open('model_architecture.json', 'r') as f:
    new_model_1 = model_from_json(f.read())
new_model_1.load_weights('model_weights.h5')

#### Save/Load the Entire Model

In [3]:
from keras.models import load_model
# Option 2: Save/load the entire model
# Creates a HDF5 file 'my_model.h5'
model.save('mnist_mlp.h5')
# Deletes the existing model
del model  
# Returns a compiled model identical to the previous one
model = load_model('mnist_mlp.h5')

### Activity: Save the MNST MLP model using one of the options above

In [ ]:
from keras.models import load_model
# Create a HDF5 file 'mnist_mlp.h5'
model.save('mnist_mlp.h5')
# Delete the existing model
del model  

## Visualizing Models in SVG images

In [4]:
from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
